In [ ]:
import pyarrow.dataset as ds

# Load the Parquet directory as a dataset
dataset = ds.dataset('../data/jane-street/lags.parquet', format="parquet")

# Convert to a Pandas DataFrame
lags = dataset.to_table().to_pandas()

# Inspect the data
print("head: ",lags.head())
print("lags: ", lags.info())


In [2]:
print(lags.columns)
print(lags.dtypes)


Index(['date_id', 'time_id', 'symbol_id', 'responder_0_lag_1',
       'responder_1_lag_1', 'responder_2_lag_1', 'responder_3_lag_1',
       'responder_4_lag_1', 'responder_5_lag_1', 'responder_6_lag_1',
       'responder_7_lag_1', 'responder_8_lag_1'],
      dtype='object')
date_id                int16
time_id                int16
symbol_id               int8
responder_0_lag_1    float32
responder_1_lag_1    float32
responder_2_lag_1    float32
responder_3_lag_1    float32
responder_4_lag_1    float32
responder_5_lag_1    float32
responder_6_lag_1    float32
responder_7_lag_1    float32
responder_8_lag_1    float32
dtype: object


In [3]:
print(lags.isnull().sum())


date_id              0
time_id              0
symbol_id            0
responder_0_lag_1    0
responder_1_lag_1    0
responder_2_lag_1    0
responder_3_lag_1    0
responder_4_lag_1    0
responder_5_lag_1    0
responder_6_lag_1    0
responder_7_lag_1    0
responder_8_lag_1    0
dtype: int64


In [4]:
print(lags.duplicated().sum())


0


In [5]:
import pandas as pd
train_data = pd.read_parquet('../data/XGBoost_data.parquet')

train_data = train_data.merge(lags, on=['date_id', 'time_id', 'symbol_id'], how='left')


In [7]:
train_data.to_parquet('../data/Lagged_Data.parquet', index=False)

In [8]:
print(train_data.isnull().sum())  # Count missing values in all columns
train_data_with_indicator = train_data.merge(
    lags, on=['date_id', 'time_id', 'symbol_id'], how='left', indicator=True
)
print(train_data_with_indicator['_merge'].value_counts())


date_id                     0
time_id                     0
symbol_id                   0
weight                      0
feature_00                  0
                       ...   
responder_4_lag_1    38914961
responder_5_lag_1    38914961
responder_6_lag_1    38914961
responder_7_lag_1    38914961
responder_8_lag_1    38914961
Length: 90, dtype: int64
_merge
left_only     38914961
right_only           0
both                 0
Name: count, dtype: int64


In [10]:
print(train_data.shape)  # Rows and columns after the merge

print(train_data.head())


(38914961, 90)
   date_id  time_id  symbol_id    weight  feature_00  feature_01  feature_04  \
0      528        0          1  3.886676    0.503752    1.338204    0.812773   
1      528        0          2  1.396185   -0.058833    0.988444   -0.027737   
2      528        0          3  0.664355   -0.282893    1.939934   -0.175234   
3      528        0          5  1.960854    0.337874    1.533237   -0.114100   
4      528        0          7  2.161128   -0.568770    1.764450    0.079021   

   feature_05  feature_06  feature_07  ...  partition_id  responder_0_lag_1  \
0   -0.095654   -1.686406   -0.975024  ...             3                NaN   
1   -0.110813   -1.686406   -1.412440  ...             3                NaN   
2   -0.072281   -1.686406   -0.977644  ...             3                NaN   
3   -0.104692   -1.686406   -1.526535  ...             3                NaN   
4   -0.107042   -1.686406   -0.973510  ...             3                NaN   

   responder_1_lag_1  respond

In [2]:
import pyarrow.dataset as ds
import pandas as pd

# Load the Parquet dataset
dataset = ds.dataset('../data/jane-street/lags.parquet', format="parquet")

# Convert to a Pandas DataFrame
lags = dataset.to_table().to_pandas()

# Convert `date_id` to a plain integer type (e.g., int16 or int32)
lags['date_id'] = lags['date_id'].astype('int16')

# Save the fixed version
lags.to_parquet('../data/jane-street/lags_fixed.parquet', index=False)


In [3]:
fixed_lags = pd.read_parquet('../data/jane-street/lags_fixed.parquet')
print(fixed_lags.dtypes)


date_id                int16
time_id                int16
symbol_id               int8
responder_0_lag_1    float32
responder_1_lag_1    float32
responder_2_lag_1    float32
responder_3_lag_1    float32
responder_4_lag_1    float32
responder_5_lag_1    float32
responder_6_lag_1    float32
responder_7_lag_1    float32
responder_8_lag_1    float32
dtype: object


In [16]:
train_data = pd.read_parquet('../data/XGBoost_data.parquet')
lags = pd.read_parquet('../data/jane-street/lags_fixed.parquet')

#train_data = train_data.merge(lags, on=['date_id', 'time_id', 'symbol_id'], how='left')


In [ ]:
print("shape: ", train_data.shape)  # Rows and columns after the merge

print("head: ", train_data.head())


In [8]:
# Check if the merge keys exist in both datasets
print(train_data.columns)
print(lags.columns)

# Check the data types of the keys
print(train_data[['date_id', 'time_id', 'symbol_id']].dtypes)
print(lags[['date_id', 'time_id', 'symbol_id']].dtypes)

# Check for unique values in the keys
print(train_data[['date_id', 'time_id', 'symbol_id']].drop_duplicates().head())
print(lags[['date_id', 'time_id', 'symbol_id']].drop_duplicates().head())


Index(['date_id', 'time_id', 'symbol_id', 'weight', 'feature_00', 'feature_01',
       'feature_04', 'feature_05', 'feature_06', 'feature_07', 'feature_08',
       'feature_09', 'feature_10', 'feature_11', 'feature_12', 'feature_13',
       'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18',
       'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23',
       'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28',
       'feature_29', 'feature_30', 'feature_32', 'feature_33', 'feature_36',
       'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41',
       'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46',
       'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51',
       'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56',
       'feature_57', 'feature_58', 'feature_59', 'feature_61', 'feature_62',
       'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_6

In [11]:
# Standardize data types explicitly
train_data['date_id'] = train_data['date_id'].astype('int16')
train_data['time_id'] = train_data['time_id'].astype('int16')
train_data['symbol_id'] = train_data['symbol_id'].astype('int8')

lags['date_id'] = lags['date_id'].astype('int16')
lags['time_id'] = lags['time_id'].astype('int16')
lags['symbol_id'] = lags['symbol_id'].astype('int8')


In [12]:
# Find common keys between train_data and lags
common_keys = pd.merge(
    train_data[['date_id', 'time_id', 'symbol_id']],
    lags[['date_id', 'time_id', 'symbol_id']],
    on=['date_id', 'time_id', 'symbol_id'],
    how='inner'
)
print(f"Number of matching keys: {len(common_keys)}")


Number of matching keys: 0


In [10]:
missing_keys = train_data.merge(
    lags[['date_id', 'time_id', 'symbol_id']],
    on=['date_id', 'time_id', 'symbol_id'],
    how='left',
    indicator=True
)
print(missing_keys['_merge'].value_counts())


_merge
left_only     38914961
right_only           0
both                 0
Name: count, dtype: int64


In [13]:
# Look for keys in train_data that are missing from lags
missing_keys = train_data.merge(
    lags[['date_id', 'time_id', 'symbol_id']],
    on=['date_id', 'time_id', 'symbol_id'],
    how='left',
    indicator=True
)
print(missing_keys[missing_keys['_merge'] == 'left_only'][['date_id', 'time_id', 'symbol_id']].head())


   date_id  time_id  symbol_id
0      528        0          1
1      528        0          2
2      528        0          3
3      528        0          5
4      528        0          7


In [14]:
# Extract the first missing key from train_data
missing_key = missing_keys[missing_keys['_merge'] == 'left_only'][['date_id', 'time_id', 'symbol_id']].iloc[0]

# Check if this key exists in lags
print(lags[
    (lags['date_id'] == missing_key['date_id']) &
    (lags['time_id'] == missing_key['time_id']) &
    (lags['symbol_id'] == missing_key['symbol_id'])
])


Empty DataFrame
Columns: [date_id, time_id, symbol_id, responder_0_lag_1, responder_1_lag_1, responder_2_lag_1, responder_3_lag_1, responder_4_lag_1, responder_5_lag_1, responder_6_lag_1, responder_7_lag_1, responder_8_lag_1]
Index: []


In [15]:
# Filter lags to the date_id range in train_data
lags = lags[lags['date_id'].isin(train_data['date_id'].unique())]

# Re-run the merge after filtering
train_data = train_data.merge(lags, on=['date_id', 'time_id', 'symbol_id'], how='left')
print(train_data.head())


   date_id  time_id  symbol_id    weight  feature_00  feature_01  feature_04  \
0      528        0          1  3.886676    0.503752    1.338204    0.812773   
1      528        0          2  1.396185   -0.058833    0.988444   -0.027737   
2      528        0          3  0.664355   -0.282893    1.939934   -0.175234   
3      528        0          5  1.960854    0.337874    1.533237   -0.114100   
4      528        0          7  2.161128   -0.568770    1.764450    0.079021   

   feature_05  feature_06  feature_07  ...  partition_id  responder_0_lag_1  \
0   -0.095654   -1.686406   -0.975024  ...             3                NaN   
1   -0.110813   -1.686406   -1.412440  ...             3                NaN   
2   -0.072281   -1.686406   -0.977644  ...             3                NaN   
3   -0.104692   -1.686406   -1.526535  ...             3                NaN   
4   -0.107042   -1.686406   -0.973510  ...             3                NaN   

   responder_1_lag_1  responder_2_lag_1  res

In [17]:
print(f"train_data date range: {train_data['date_id'].min()} to {train_data['date_id'].max()}")
print(f"lags date range: {lags['date_id'].min()} to {lags['date_id'].max()}")


train_data date range: 528 to 1698
lags date range: 0 to 0


In [18]:
lags.head()

,date_id,time_id,symbol_id,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
0,0,0,0,-0.442215,-0.322407,0.143594,-0.926890,-0.782236,-0.036595,-1.305746,-0.795677,-0.143724
1,0,0,1,-0.651829,-1.707840,-0.893942,-1.065488,-1.871338,-0.615652,-1.162801,-1.205924,-1.245934
2,0,0,2,-0.656373,-0.264575,-0.892879,-1.511886,-1.033480,-0.378265,-1.574290,-1.863071,-0.027343
3,0,0,3,-0.188186,-0.190970,-0.701490,0.098453,-1.015506,-0.054984,0.329152,-0.965471,0.576635
4,0,0,4,-0.257462,-0.471325,-0.297420,0.074018,-0.324194,-0.597093,0.219856,-0.276356,-0.904790
